In [2]:
import http.client
import json
import time
import csv
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)

        
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')   

In [3]:
fromDate ='12/18/2019'
toDate   = '1/13/2021'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_ftmcmry = pd.DataFrame()
df_lthbrg  = pd.DataFrame()
df_mdcnht  = pd.DataFrame()
df_rddr    = pd.DataFrame()
df_slvlk   = pd.DataFrame()
df_list = {}

streams = [242498, 242497, 242500, 242508, 242511, 242519, 242522]
city_names = ['calgary', 'edmonton', 'ftmcmry','lthbrg','mdcnht', 'rddr','slvlk']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]
df_ftmcmry = df_list[2]
df_lthbrg  = df_list[3]
df_mdcnht  = df_list[4]
df_rddr    = df_list[5]
df_slvlk   = df_list[6]


alberta_weather_merged = pd.concat([df_clgry, df_edmtn, df_ftmcmry, 
                                   df_lthbrg, df_mdcnht,
                                   df_rddr, df_slvlk], axis=1)

alberta_weather_merged = alberta_weather_merged[['Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk']]
alberta_weather_merged

2021-01-13 01:51:30.289842 Outputing stream /api/StreamData/242498?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:33.457273 Outputing stream /api/StreamData/242497?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:36.302775 Outputing stream /api/StreamData/242500?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:39.673894 Outputing stream /api/StreamData/242508?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:42.569725 Outputing stream /api/StreamData/242511?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:45.436278 Outputing stream /api/StreamData/242519?fromDate=12/18/2019&toDate=1/13/2021 res code 200
2021-01-13 01:51:48.402163 Outputing stream /api/StreamData/242522?fromDate=12/18/2019&toDate=1/13/2021 res code 200


,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Dec 18 2019 00:00,-1,20,1,15,-7,20,2,50,1,40,-1,20,-4,30
1,Dec 18 2019 01:00,-2,20,0.0000,15,-8,20,1,50,1,40,-1,20,-4,50
2,Dec 18 2019 02:00,-2,20,-1,15,-10,20,1,50,1,40,-2,20,-5,20
3,Dec 18 2019 03:00,-2,15,-2,15,-11,20,1,50,1,20,-3,20,-6,20
4,Dec 18 2019 04:00,-1,15,-4,15,-13,20,0.0000,50,0.0000,20,-3,5,-7,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403,Jan 12 2021 19:00,6,30,-2,10,-1,20,10,60,6,30,1,20,-2,10
9404,Jan 12 2021 20:00,5,30,-3,10,-1,20,10,60,6,30,0.0000,10,-2,10
9405,Jan 12 2021 21:00,6,30,-3,10,-2,10,10,30,7,50,-1,10,-5,10
9406,Jan 12 2021 22:00,5,15,-4,10,-2,10,10,30,6,30,-3,10,-6,10


In [4]:
df1 = pd.read_csv('alberta_weather_all_v1.csv')
df1

,Unnamed: 0,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,...,MA_tempcalgary,MA_tempedmonton,MA_tempftmcmry,MA_tempmdcnht,MA_temprddr,MA_tempslvlk,average_temp,average_temp_squared,average_temp_cube,MA_temp_avg
0,0,Jan 01 2015 00:00,-5.5,6.0,-7.5,NaN,-12.4,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-5.016667,25.166944,-126.254171,NaN
1,1,Jan 01 2015 01:00,-2.5,2.0,-7.6,NaN,-13.0,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.883333,15.080278,-58.561745,NaN
2,2,Jan 01 2015 02:00,-2.6,4.0,-7.9,NaN,-13.6,4,-3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.814286,14.548776,-55.493187,NaN
3,3,Jan 01 2015 03:00,-1.6,13.0,-7.8,NaN,-13.5,4,-2.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.200000,10.240000,-32.768000,NaN
4,4,Jan 01 2015 04:00,-1.7,10.0,-7.6,NaN,-13.7,3,-2.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.142857,9.877551,-31.043732,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42115,42115,Dec 17 2019 19:00,4.7,9.0,-1.1,9.0,-4.9,16,3.4,43.0,...,2.870833,-1.541667,-16.029167,0.275000,-5.129167,-8.837500,-0.085714,0.007347,-0.000630,-3.764286
42116,42116,Dec 17 2019 20:00,3.1,17.0,-2.5,15.0,-4.9,10,1.2,23.0,...,2.979167,-1.633333,-15.283333,0.600000,-4.675000,-8.720833,-0.542857,0.294694,-0.159977,-3.518452
42117,42117,Dec 17 2019 21:00,6.8,17.0,-3.2,14.0,-6.4,9,2.4,39.0,...,3.216667,-1.720833,-14.637500,0.912500,-4.254167,-8.154167,1.414286,2.000204,2.828860,-3.204167
42118,42118,Dec 17 2019 22:00,6.9,17.0,-3.7,13.0,-4.2,9,2.7,34.0,...,3.470833,-1.775000,-13.929167,1.137500,-3.908333,-7.604167,1.314286,1.727347,2.270227,-2.897619


In [5]:
df1.columns

Index(['Unnamed: 0', 'Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk', 'temp_calgary_squared',
       'temp_edmonton_squared', 'temp_ftmcmry_squared', 'temp_lthbrg_squared',
       'temp_mdcnht_squared', 'temp_rddr_squared', 'temp_slvlk_squared',
       'temp_calgary_cube', 'temp_edmonton_cube', 'temp_ftmcmry_cube',
       'temp_lthbrg_cube', 'temp_mdcnht_cube', 'temp_rddr_cube',
       'temp_slvlk_cube', 'wind_chill_index_calgary',
       'wind_chill_index_edmonton', 'wind_chill_index_ftmcmry',
       'wind_chill_index_lthbrg', 'wind_chill_index_mdcnht',
       'wind_chill_index_rddr', 'wind_chill_index_slvlk', 'MA_tempcalgary',
       'MA_tempedmonton', 'MA_tempftmcmry', 'MA_tempmdcnht', 'MA_temprddr',
       'MA_tempslvlk', 'average_temp', 'average_temp_squared',
       'average_temp

In [6]:
df1 = df1[['Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk']]
df1

,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Jan 01 2015 00:00,-5.5,6.0,-7.5,NaN,-12.4,6,NaN,NaN,-6.1,NaN,0.3,11.0,1.1,32.0
1,Jan 01 2015 01:00,-2.5,2.0,-7.6,NaN,-13.0,3,NaN,NaN,-5.1,NaN,3.3,20.0,1.6,22.0
2,Jan 01 2015 02:00,-2.6,4.0,-7.9,NaN,-13.6,4,-3.5,NaN,-3.8,NaN,3.3,21.0,1.4,18.0
3,Jan 01 2015 03:00,-1.6,13.0,-7.8,NaN,-13.5,4,-2.6,NaN,-1.9,NaN,3.4,22.0,1.6,17.0
4,Jan 01 2015 04:00,-1.7,10.0,-7.6,NaN,-13.7,3,-2.6,NaN,0.1,11.0,3.0,16.0,0.5,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42115,Dec 17 2019 19:00,4.7,9.0,-1.1,9.0,-4.9,16,3.4,43.0,2.7,26.0,0.4,8.0,-5.8,15.0
42116,Dec 17 2019 20:00,3.1,17.0,-2.5,15.0,-4.9,10,1.2,23.0,3.4,29.0,2.0,11.0,-6.1,9.0
42117,Dec 17 2019 21:00,6.8,17.0,-3.2,14.0,-6.4,9,2.4,39.0,3.7,54.0,1.7,13.0,4.9,32.0
42118,Dec 17 2019 22:00,6.9,17.0,-3.7,13.0,-4.2,9,2.7,34.0,2.9,37.0,0.0,8.0,4.6,45.0


In [7]:
weather_db = pd.concat([df1, alberta_weather_merged], ignore_index=True)
weather_db

,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Jan 01 2015 00:00,-5.5,6,-7.5,NaN,-12.4,6,NaN,NaN,-6.1,NaN,0.3,11,1.1,32
1,Jan 01 2015 01:00,-2.5,2,-7.6,NaN,-13,3,NaN,NaN,-5.1,NaN,3.3,20,1.6,22
2,Jan 01 2015 02:00,-2.6,4,-7.9,NaN,-13.6,4,-3.5,NaN,-3.8,NaN,3.3,21,1.4,18
3,Jan 01 2015 03:00,-1.6,13,-7.8,NaN,-13.5,4,-2.6,NaN,-1.9,NaN,3.4,22,1.6,17
4,Jan 01 2015 04:00,-1.7,10,-7.6,NaN,-13.7,3,-2.6,NaN,0.1,11,3,16,0.5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51523,Jan 12 2021 19:00,6,30,-2,10,-1,20,10,60,6,30,1,20,-2,10
51524,Jan 12 2021 20:00,5,30,-3,10,-1,20,10,60,6,30,0.0000,10,-2,10
51525,Jan 12 2021 21:00,6,30,-3,10,-2,10,10,30,7,50,-1,10,-5,10
51526,Jan 12 2021 22:00,5,15,-4,10,-2,10,10,30,6,30,-3,10,-6,10


In [9]:
weather_db.columns

Index(['Datetime_calgary', 'temp_calgary', 'wind_calgary', 'temp_edmonton',
       'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry', 'temp_lthbrg',
       'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr', 'wind_rddr',
       'temp_slvlk', 'wind_slvlk'],
      dtype='object')

In [19]:
import sqlite3
conn = sqlite3.connect('weather_db.db')
c = conn.cursor()
c.execute("""
          CREATE TABLE HISTORICALFCAST (Datetime_calgary,temp_calgary, wind_calgary,
          temp_edmonton,wind_edmonton, temp_ftmcmry, wind_ftmcmry, temp_lthbrg,
          wind_lthbrg,temp_mdcnht, wind_mdcnht, temp_rddr, wind_rddr,temp_slvlk, wind_slvlk)
          """
)
conn.commit()

In [20]:
weather_db.to_sql('HISTORICALFCAST', conn, if_exists='replace', index = False)

In [21]:
c.close()
conn.close()

In [22]:
conn = sqlite3.connect('weather_db.db')

In [23]:
fromDate ='1/13/2021'
toDate   ='1/13/2021'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_ftmcmry = pd.DataFrame()
df_lthbrg  = pd.DataFrame()
df_mdcnht  = pd.DataFrame()
df_rddr    = pd.DataFrame()
df_slvlk   = pd.DataFrame()
df_list = {}

streams = [242498, 242497, 242500, 242508, 242511, 242519, 242522]
city_names = ['calgary', 'edmonton', 'ftmcmry','lthbrg','mdcnht', 'rddr','slvlk']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]
df_ftmcmry = df_list[2]
df_lthbrg  = df_list[3]
df_mdcnht  = df_list[4]
df_rddr    = df_list[5]
df_slvlk   = df_list[6]


alberta_weather_merged = pd.concat([df_clgry, df_edmtn, df_ftmcmry, 
                                   df_lthbrg, df_mdcnht,
                                   df_rddr, df_slvlk], axis=1)

alberta_weather_merged = alberta_weather_merged[['Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk']]
alberta_weather_merged

2021-01-13 02:31:53.401222 Outputing stream /api/StreamData/242498?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:31:54.830608 Outputing stream /api/StreamData/242497?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:31:56.306016 Outputing stream /api/StreamData/242500?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:31:57.732532 Outputing stream /api/StreamData/242508?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:31:59.134841 Outputing stream /api/StreamData/242511?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:32:00.600018 Outputing stream /api/StreamData/242519?fromDate=1/13/2021&toDate=1/13/2021 res code 200
2021-01-13 02:32:02.028766 Outputing stream /api/StreamData/242522?fromDate=1/13/2021&toDate=1/13/2021 res code 200


,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Jan 13 2021 00:00,2,15,-3,10,-3,10,9,30,6,30,-2,10,-6,10
1,Jan 13 2021 01:00,2,15,-1,10,-3,10,9,30,6,30,-1,10,-4,10
2,Jan 13 2021 02:00,1,15,0.0000,10,-3,10,8,30,7,30,1,10,-3,10
3,Jan 13 2021 03:00,2,15,0.0000,10,-4,10,9,30,7,30,2,30,-3,10
4,Jan 13 2021 04:00,3,30,1,10,-4,10,10,60,7,30,2,30,-3,10
5,Jan 13 2021 05:00,4,30,1,10,-5,10,11,60,7,30,3,30,-3,10
6,Jan 13 2021 06:00,4,30,1,10,-4,10,11,60,7,30,3,30,-2,10
7,Jan 13 2021 07:00,5,30,2,10,-4,10,12,60,8,30,3,30,-2,10
8,Jan 13 2021 08:00,5,30,2,10,-3,10,12,60,8,70,3,70,-1,10
9,Jan 13 2021 09:00,4,50,2,50,-3,10,11,80,9,70,2,70,-1,40


In [24]:
alberta_weather_merged.to_sql('HISTORICALFCAST', conn, if_exists='append',index= False)

In [30]:
c = conn.cursor()
c.execute('''  
SELECT * FROM HISTORICALFCAST
          ''')
results = c.fetchall()
tester_df = pd.DataFrame(results)
tester_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Jan 01 2015 00:00,-5.5,6.0,-7.5,None,-12.4,6,None,None,-6.1,None,0.3,11.0,1.1,32.0
1,Jan 01 2015 01:00,-2.5,2.0,-7.6,None,-13.0,3,None,None,-5.1,None,3.3,20.0,1.6,22.0
2,Jan 01 2015 02:00,-2.6,4.0,-7.9,None,-13.6,4,-3.5,None,-3.8,None,3.3,21.0,1.4,18.0
3,Jan 01 2015 03:00,-1.6,13.0,-7.8,None,-13.5,4,-2.6,None,-1.9,None,3.4,22.0,1.6,17.0
4,Jan 01 2015 04:00,-1.7,10.0,-7.6,None,-13.7,3,-2.6,None,0.1,11.0,3.0,16.0,0.5,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51547,Jan 13 2021 19:00,0.0000,20,0.0000,30,-1,20,1,30,0.0000,50,-2,30,-2,40
51548,Jan 13 2021 20:00,-1,20,0.0000,30,-2,20,1,30,0.0000,50,-2,30,-2,40
51549,Jan 13 2021 21:00,-1,30,0.0000,20,-3,20,1,40,0.0000,30,-3,30,-2,40
51550,Jan 13 2021 22:00,-1,30,-1,20,-3,20,0.0000,40,0.0000,30,-3,30,-2,40


In [37]:
c.close()
conn.close()